In [8]:
import os
import gtfs_kit as gk
import pickle
import pandas as pd
import osmnx as ox
from tqdm import tqdm

import sys
import datetime as dt
import importlib
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import Row, SparkSession
from pyspark.sql import functions as F
from pyspark import SparkConf
import numpy as np
import pickle
import joblib
import json
import random
import matplotlib.pyplot as plt
import matplotlib as mpl
from copy import deepcopy
from tqdm import trange, tqdm

mpl.rcParams['figure.facecolor'] = 'white'

import warnings

import pandas as pd
import swifter
pd.set_option('display.max_columns', None)
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

import pyspark
print(pyspark.__version__)

3.3.0


In [2]:
spark = SparkSession.builder.config('spark.executor.cores', '8').config('spark.executor.memory', '80g')\
        .config("spark.sql.session.timeZone", "UTC").config('spark.driver.memory', '40g').master("local[26]")\
        .appName("wego-daily").config('spark.driver.extraJavaOptions', '-Duser.timezone=UTC').config('spark.executor.extraJavaOptions', '-Duser.timezone=UTC')\
        .config("spark.sql.datetime.java8API.enabled", "true").config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .config("spark.sql.autoBroadcastJoinThreshold", -1)\
        .config("spark.driver.maxResultSize", 0)\
        .config("spark.shuffle.spill", "true")\
        .getOrCreate()
f = os.path.join('/home/jptalusan/mta_stationing_problem/data/processed/apc_weather_gtfs.parquet')
apcdata = spark.read.load(f)
todelete = apcdata.filter('(load < 0) OR (load IS NULL)').select('transit_date','trip_id','overload_id').distinct()
todelete=todelete.withColumn('marker',F.lit(1))

#joining and whereever the records are not found in sync error table the marker will be null
apcdataafternegdelete=apcdata.join(todelete,on=['trip_id','transit_date','overload_id'],how='left').filter('marker is null').drop('marker')
apcdataafternegdelete = apcdataafternegdelete.sort(['trip_id', 'overload_id'])

apcdataafternegdelete.createOrReplaceTempView("apc")

# filter subset
get_columns = ['vehicle_id', 'vehicle_capacity']
get_str = ", ".join([c for c in get_columns])

query = f"""
SELECT {get_str}
FROM apc
"""
print(query)

apcdataafternegdelete = spark.sql(query)
apcdataafternegdelete = apcdataafternegdelete.dropDuplicates(['vehicle_id', 'vehicle_capacity'])
vehicle_capacity_df = apcdataafternegdelete.toPandas()

22/10/01 20:10:45 WARN Utils: Your hostname, scope-vanderbilt resolves to a loopback address: 127.0.1.1; using 10.2.218.69 instead (on interface enp8s0)
22/10/01 20:10:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/01 20:10:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/01 20:10:50 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.

SELECT vehicle_id, vehicle_capacity
FROM apc



In [3]:
# Vehicles and vehicle capacities
# Set nan to 40.0 for now
vehicle_capacity_df['vehicle_capacity'] = vehicle_capacity_df['vehicle_capacity'].fillna(40.0)
print(vehicle_capacity_df.vehicle_capacity.unique())
print(vehicle_capacity_df.vehicle_capacity.value_counts())

[40. 45. 55. 15.]
40.0    161
45.0     38
55.0     22
15.0      3
Name: vehicle_capacity, dtype: int64


# Generate the scenarios for the simulator
* Each scenario requires N files:
    * vehicle config
    * trip config
* Each scenario needs corresponding distribution models:
    * passenger boarding
    * passenger occupancy
    * travel times

## Baseline
* Uses the same vehicle and route combinations as actual APC data

In [4]:
schedule_date = '2021-08-23'

In [5]:
# filter subset
get_columns = ['trip_id', 'transit_date', 'departure_time', 'scheduled_time', 'route_id', 'route_direction_name',
               'block_abbr', 'block_stop_order', 'stop_sequence', 'stop_id_original', 'vehicle_id', 'vehicle_capacity', 'zero_load_at_trip_end']
get_str = ", ".join([c for c in get_columns])

query = f"""
SELECT {get_str}
FROM apc
WHERE (transit_date == '{schedule_date}')
"""
print(query)

apcdataafternegdelete = spark.sql(query)
apcdataafternegdelete = apcdataafternegdelete.na.drop(subset='departure_time')
apcdataafternegdelete = apcdataafternegdelete.na.fill(value=0,subset=["zero_load_at_trip_end"])
df = apcdataafternegdelete.toPandas()


SELECT trip_id, transit_date, departure_time, scheduled_time, route_id, route_direction_name, block_abbr, block_stop_order, stop_sequence, stop_id_original, vehicle_id, vehicle_capacity, zero_load_at_trip_end
FROM apc
WHERE (transit_date == '2021-08-23')



In [6]:
selected_df = df

In [18]:
df.query("trip_id == '246300'").sort_values('stop_sequence')

,trip_id,transit_date,departure_time,scheduled_time,route_id,route_direction_name,block_abbr,block_stop_order,stop_sequence,stop_id_original,vehicle_id,vehicle_capacity,zero_load_at_trip_end
28183,246300,2021-08-23,2021-08-23 15:49:48,2021-08-23 15:48:00,86,FROM NASHVILLE,8602,4,1,MCC4_23,1200,55.0,0
28190,246300,2021-08-23,2021-08-23 15:51:00,2021-08-23 15:50:40,86,FROM NASHVILLE,8602,5,2,6AVDEASN,1200,55.0,0
28188,246300,2021-08-23,2021-08-23 15:52:26,2021-08-23 15:51:59,86,FROM NASHVILLE,8602,6,3,6AVCHUSN,1200,55.0,0
28181,246300,2021-08-23,2021-08-23 15:55:00,2021-08-23 15:54:52,86,FROM NASHVILLE,8602,7,4,8ABROSN,1200,55.0,0
28191,246300,2021-08-23,2021-08-23 15:56:52,2021-08-23 15:56:11,86,FROM NASHVILLE,8602,8,5,8ADEMSN,1200,55.0,0
28184,246300,2021-08-23,2021-08-23 15:58:10,2021-08-23 15:59:43,86,FROM NASHVILLE,8602,9,6,LAFEWIEN,1200,55.0,0
28182,246300,2021-08-23,2021-08-23 15:59:02,2021-08-23 16:01:00,86,FROM NASHVILLE,8602,10,7,LAF5AEN,1200,55.0,0
28180,246300,2021-08-23,2021-08-23 16:35:36,2021-08-23 16:46:54,86,FROM NASHVILLE,8602,11,8,MURCITEN,1200,55.0,0
28193,246300,2021-08-23,2021-08-23 16:37:30,2021-08-23 16:47:46,86,FROM NASHVILLE,8602,12,9,MURJACEN,1200,55.0,0
28185,246300,2021-08-23,2021-08-23 16:39:20,2021-08-23 16:48:14,86,FROM NASHVILLE,8602,13,10,MURSTOEN,1200,55.0,0


In [10]:


os.chdir("/home/jptalusan/gits/mta_simulator_redo/data_generation")

In [11]:
# Create a dict of {[block: {trip_ids:[]}, 'block'....]}
# trip_id dict = {'route_id', route_direction_name', 'stop_id':[], 'schedule_time':[]}
# Use block as grouper in baseline
overall_block_plan = {}
for block_abbr, block_df in selected_df.groupby('block_abbr'):
    block_df = block_df.dropna(subset=['departure_time']).sort_values(['scheduled_time'])
    trip_ids = block_df.trip_id.unique().tolist()
    start_time = block_df[block_df['trip_id'] == trip_ids[0]].iloc[0]['scheduled_time'].strftime('%Y-%m-%d %H:%M:%S')
    end_time = block_df[block_df['trip_id'] == trip_ids[-1]].iloc[-1]['scheduled_time'].strftime('%Y-%m-%d %H:%M:%S')
    overall_block_plan[block_abbr] = {'trip_ids': trip_ids,
                                      'start_time': start_time,
                                      'end_time': end_time}

overall_trip_plan = {}
for trip_id, trip_df in selected_df.groupby('trip_id'):
    trip_df = trip_df.dropna(subset=['departure_time']).sort_values(['scheduled_time'])
    route_id = trip_df.iloc[0].route_id.tolist()
    route_direction = trip_df.iloc[0].route_direction_name
    zero_load_at_trip_end = trip_df.iloc[-1].zero_load_at_trip_end.tolist()
    scheduled_time = trip_df.scheduled_time.dt.strftime('%Y-%m-%d %H:%M:%S').tolist()
    stop_sequence = trip_df.stop_sequence.tolist()
    stop_sequence = list(range(0, len(stop_sequence)))
    # stop_sequence = [ss - 1 for ss in stop_sequence]
    stop_id_original = trip_df.stop_id_original.tolist()
    
    overall_trip_plan[trip_id] = {'route_id': route_id, 
                                  'route_direction': route_direction, 
                                  'scheduled_time': scheduled_time, 
                                  'stop_sequence': stop_sequence, 
                                  'stop_id_original': stop_id_original,
                                  'zero_load_at_trip_end':zero_load_at_trip_end,
                                  'last_stop_sequence': stop_sequence[-1],
                                  'last_stop_id': stop_id_original[-1]}

len(overall_trip_plan), len(overall_block_plan)

with open('results/trip_plan.json', 'w') as fp:
    json.dump(overall_trip_plan, fp, sort_keys=True, indent=2)

In [12]:
# Vehicle assignments
# Each vehicle config is a dict: {vehicle_capacity, blocks}
DEFAULT_CAPACITY = 40.0
overall_vehicle_plan = {}

for vehicle_id, vehicle_df in selected_df.groupby('vehicle_id'):
    vehicle_df = vehicle_df.dropna(subset=['departure_time']).sort_values(['scheduled_time'])
    vehicle_capacity = vehicle_df.iloc[0].vehicle_capacity
    if np.isnan(vehicle_capacity):
        vehicle_capacity = DEFAULT_CAPACITY
    # TODO: This is not the baseline behavior
    starting_depot = 'MCC5_1'
    service_type = 'regular'
    blocks = [block for block in vehicle_df.block_abbr.unique().tolist()]
    trips = []
    for block in blocks:
        block_df = vehicle_df.query("block_abbr == @block")
        for trip in block_df.trip_id.unique().tolist():
            trips.append((str(block), str(trip)))
    overall_vehicle_plan[vehicle_id] = {'vehicle_capacity': vehicle_capacity, 'trips': trips, 'starting_depot': starting_depot, 'service_type': service_type}
    
len(overall_vehicle_plan)
    
with open('results/vehicle_plan.json', 'w') as fp:
    json.dump(overall_vehicle_plan, fp, sort_keys=True, indent=2)

## Set (based on GTFS?)
* You provide a scheduled date.
* You provide a list of vehicles and blocks.
* Must generate the same format as baseline.
* Must check whether trips are valid (start should not be before end of previous trip/block).
> Why is the GTFS block_id different from the APC data block?

In [9]:
schedule_date = '2021823'
fp = '/media/seconddrive/wego-occupancy-JP/data/static_gtfs/WeGoRawGTFS/04-october-2021-fixed.zip'
feed = gk.read_feed(fp, dist_units='km')
feed.validate()
gk.get_routes(feed).head(1)
tdf = gk.get_trips(feed, date=schedule_date)
# tdf.query("block_id == '27717'")
# tdf.block_id.unique()
gk.get_stop_times(feed, date=schedule_date).query("trip_id == '259306'").head()
gk.get_start_and_end_times(feed, date=schedule_date)

('04:20:00', '25:10:00')

In [64]:
tdf

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,14,1,259283,BUENA VISTA,NaN,0,27711,16555,1,1
1,14,1,259284,BUENA VISTA,NaN,0,27711,16555,1,1
2,14,1,259285,BUENA VISTA,NaN,0,27711,16555,1,1
3,14,1,259286,BUENA VISTA,NaN,0,27711,16555,1,1
4,14,1,259287,BUENA VISTA,NaN,0,27711,16555,1,1
...,...,...,...,...,...,...,...,...,...,...
2061,95,1,264831,FRANKLIN SPRING HILL,NaN,0,28214,16716,2,2
2062,95,1,264832,FRANKLIN SPRING HILL VIA VANDERBILT,NaN,0,28213,16717,2,2
2063,95,1,264833,DOWNTOWN NASHVILLE,NaN,1,28218,16718,2,2
2064,95,1,264834,DOWNTOWN NASHVILLE,NaN,1,28217,16718,2,2


In [11]:
# For now just randomly assign trips and capacities etc...

## Random

## Process scenarios
* Generate the distributions based on these scenarios.
* Sample occupancy and boarding every time a bus reaches a stop (in trip_plan)
* Sample travel time every time a bus leaves a stop (trip_plan)
* Distance traveled is static: `gtfs_distance_pairs_km.pkl`
* Sample disruption from static probabilities: `disruption_probabilities.pkl`

In [67]:
with open('code/Scenarios/baseline/config.json') as f:
    d = json.load(f)
schedule_date = d['schedule_date']
isWeekend = int(pd.Timestamp(schedule_date).weekday() >= 5)

In [68]:
occupancy_dataset = pd.read_pickle('code/Scenarios/data/loads_15min_processed_dataset.pkl')
boarding_dataset = pd.read_pickle('code/Scenarios/data/ons_60min_processed_dataset.pkl')
traveltime_dataset = pd.read_pickle('code/Scenarios/data/travel_time_15min_processed_dataset.pkl')
disruption_dataset = pd.read_pickle('code/Scenarios/data/disruption_probabilities.pkl')

In [69]:
with open('code/Scenarios/baseline/block_plan.pkl', 'rb') as file:
    overall_block_plan = pickle.load(file)

In [70]:
with open('code/Scenarios/baseline/block_plan.pkl', 'rb') as file:
    overall_block_plan = pickle.load(file)
with open('code/Scenarios/baseline/trip_plan.pkl', 'rb') as file:
    overall_trip_plan = pickle.load(file)
with open('code/Scenarios/baseline/vehicle_plan.pkl', 'rb') as file:
    overall_vehicle_plan = pickle.load(file)

In [71]:
# dist = self.data.loc[(route_id_dir, block_abbr, stop_sequence, stop_id, time_window, is_weekend)]['time_to_next_stop']

In [111]:
random_seed = 100
np.random.seed(random_seed)
random.seed(random_seed)

sampled_distributions = []
for block_abbr in tqdm(overall_block_plan):
    trip_ids = overall_block_plan[block_abbr]['trip_ids']
    for trip in trip_ids:
        trip_data = overall_trip_plan[trip]
        
        scheduled_times = trip_data['scheduled_time']
        route_id = trip_data['route_id']
        route_dir = trip_data['route_direction']
        route_id_dir = str(route_id) + "_" + route_dir
        stop_id_original = trip_data['stop_id_original']
        stop_sequence = trip_data['stop_sequence']
        
        tts = []
        onss = []
        loads = []
        disruptions = []
        for i in range(len(stop_sequence)):
            minute = pd.Timestamp(scheduled_times[i]).minute
            hour = pd.Timestamp(scheduled_times[i]).hour
            
            # Travel Time distribution sampling
            TIMEWINDOW = 15
            time_window = int(minute // TIMEWINDOW + hour * (60 // TIMEWINDOW))
            try:
                tt = traveltime_dataset.loc[(route_id_dir, block_abbr, stop_sequence[i], stop_id_original[i], time_window, isWeekend)]['time_to_next_stop']
            except:
                tt = traveltime_dataset.loc[(route_id_dir, block_abbr)].explode('time_to_next_stop').query("IsWeekend == @isWeekend").sample(1, random_state=random_seed)['time_to_next_stop']
            tt = np.array(tt)
            try:
                tts.append(np.random.choice(tt[tt >= 0]))
            except:
                print("TT: ", route_id_dir, block_abbr, stop_sequence[i], stop_id_original[i], time_window, isWeekend, tt)
                tts.append(30)
            
            # Occupancy distribution sampling
            try:
                load = occupancy_dataset.loc[(route_id_dir, block_abbr, stop_id_original[i], time_window, isWeekend)]['load']
            except:
                load = occupancy_dataset.loc[(route_id_dir, block_abbr)].explode('load').query("IsWeekend == @isWeekend").sample(1, random_state=random_seed)['load']
            load = np.array(load)
            try:
                loads.append(np.random.choice(load[load >= 0]))
            except:
                print(loads)
                loads.append(0)
            
            # Boarding distribution sampling
            TIMEWINDOW = 60
            time_window = int(minute // TIMEWINDOW + hour * (60 // TIMEWINDOW))
            try:
                ons = boarding_dataset.loc[(route_id_dir, block_abbr, stop_id_original[i], time_window, isWeekend)]['ons']
            except:
                ons = boarding_dataset.loc[(route_id_dir, block_abbr)].explode('ons').query("IsWeekend == @isWeekend").sample(1, random_state=random_seed)['ons']
            ons = np.array(ons)
            try:
                onss.append(np.random.choice(ons[ons >= 0]))
            except:
                print(onss)
                onss.append(0)
            
            # Disruption probability sampling
            stop_id = stop_id_original[i]
            p = disruption_dataset.query('stop_id_original == @stop_id').iloc[0]['probability']
            disruption = np.random.choice([True, False], p=[p, 1-p])
            # disruption = False
            disruptions.append(disruption)
        
        # Create dataframe
        trip_data['block_abbr'] = block_abbr
        trip_data['trip_id'] = trip
        trip_data['travel_times'] = tts
        trip_data['ons'] = onss
        trip_data['loads'] = loads
        trip_data['disruptions'] = disruptions
        trip_data = pd.DataFrame(trip_data)
        sampled_distributions.append(trip_data)
        
sampled_distributions = pd.concat(sampled_distributions)

 41%|████      | 37/90 [00:22<00:40,  1.32it/s]

TT:  23_TO DOWNTOWN 2310 1 DWMRT 90 0 [-149.]


 46%|████▌     | 41/90 [00:26<00:36,  1.33it/s]

TT:  25_NORTHBOUND 2503 8 CHE1AVWF 54 0 [-1. -1. -1.]


 47%|████▋     | 42/90 [00:26<00:33,  1.42it/s]

TT:  25_NORTHBOUND 2503 8 CHE1AVWF 72 0 [-1. -1. -1.]


100%|██████████| 90/90 [00:41<00:00,  2.15it/s]


In [112]:
# All vehicles just read from this dataframe of pre-computed distributions
# fp = os.path.join('code/Scenarios/baseline/sampled_distributions.pkl')
fp = os.path.join('code/Scenarios/baseline/sampled_distributions_with_disruptions.pkl')
sampled_distributions.to_pickle(fp)

In [117]:
sampled_distributions.query('disruptions == True')

,route_id,route_direction,scheduled_time,stop_sequence,stop_id_original,zero_load_at_trip_end,block_abbr,trip_id,travel_times,ons,loads,disruptions
20786,52,FROM DOWNTOWN,2021-08-23 15:56:30,5,MXOFULTN,0,5205,244524,156.0,0.0,16.0,True


In [80]:
sampled_distributions.query("trip_id == '244688'").head()

,route_id,route_direction,scheduled_time,stop_sequence,stop_id_original,zero_load_at_trip_end,block_abbr,trip_id,travel_times,ons,loads,disruptions
0,55,FROM DOWNTOWN,2021-08-23 19:35:00,1,MCC4_15,0,5509,244688,781.0,15.0,42.0,False
1,55,FROM DOWNTOWN,2021-08-23 19:36:29,2,4AVCHUSN,0,5509,244688,214.0,0.0,37.0,False
2,55,FROM DOWNTOWN,2021-08-23 19:38:33,3,MXOMCCTR,0,5509,244688,150.0,3.0,43.0,False
3,55,FROM DOWNTOWN,2021-08-23 19:39:59,4,NXOPBODY,0,5509,244688,26.0,0.0,17.0,False
4,55,FROM DOWNTOWN,2021-08-23 19:41:13,5,MXOFULTN,0,5509,244688,110.0,0.0,18.0,False


In [113]:
# All vehicles just read from this dataframe of pre-computed distributions
fp = os.path.join('code/Scenarios/baseline/sampled_distributions_with_disruptions.pkl')
sampled_distributions = pd.read_pickle(fp)

In [114]:
sampled_distributions = sampled_distributions.reset_index(drop=True)
sampled_distributions

,route_id,route_direction,scheduled_time,stop_sequence,stop_id_original,zero_load_at_trip_end,block_abbr,trip_id,travel_times,ons,loads,disruptions
0,3,FROM DOWNTOWN,2021-08-23 05:20:00,1,MCC5_5,0,300,245109,398.0,9.0,2.0,False
1,3,FROM DOWNTOWN,2021-08-23 05:22:36,2,6AVDEASN,0,300,245109,82.0,12.0,9.0,False
2,3,FROM DOWNTOWN,2021-08-23 05:23:17,3,6AVCHUSN,0,300,245109,100.0,0.0,7.0,False
3,3,FROM DOWNTOWN,2021-08-23 05:24:48,4,8ABROSN,0,300,245109,132.0,0.0,10.0,False
4,3,FROM DOWNTOWN,2021-08-23 05:25:15,5,BRO9AWF,0,300,245109,56.0,0.0,7.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
28438,50,FROM DOWNTOWN,2021-08-23 23:39:18,20,ANNPREEF,0,9975,246345,74.0,0.0,0.0,False
28439,50,FROM DOWNTOWN,2021-08-23 23:40:00,21,CXOANNEX,0,9975,246345,168.0,0.0,1.0,False
28440,50,FROM DOWNTOWN,2021-08-23 23:41:45,22,CXONVLWS,0,9975,246345,110.0,0.0,1.0,False
28441,50,FROM DOWNTOWN,2021-08-23 23:44:39,23,CXODVDSN,0,9975,246345,220.0,0.0,0.0,False


In [115]:
sampled_distributions[sampled_distributions["block_abbr"] == 5205]

,route_id,route_direction,scheduled_time,stop_sequence,stop_id_original,zero_load_at_trip_end,block_abbr,trip_id,travel_times,ons,loads,disruptions
20782,52,FROM DOWNTOWN,2021-08-23 15:50:00,1,MCC4_19,0,5205,244524,252.0,8.0,18.0,False
20783,52,FROM DOWNTOWN,2021-08-23 15:51:47,2,4AVCHUSN,0,5205,244524,212.0,0.0,8.0,False
20784,52,FROM DOWNTOWN,2021-08-23 15:53:51,3,MXOMCCTR,0,5205,244524,172.0,1.0,18.0,False
20785,52,FROM DOWNTOWN,2021-08-23 15:55:16,4,NXOPBODY,0,5205,244524,54.0,1.0,12.0,False
20786,52,FROM DOWNTOWN,2021-08-23 15:56:30,5,MXOFULTN,0,5205,244524,156.0,0.0,16.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
20987,52,TO DOWNTOWN,2021-08-23 21:29:20,32,NXICHSNT,0,5205,244590,52.0,0.0,12.0,False
20988,52,TO DOWNTOWN,2021-08-23 21:31:21,33,NXILAFYT,0,5205,244590,40.0,0.0,11.0,False
20989,52,TO DOWNTOWN,2021-08-23 21:32:43,34,MXIFULTN,0,5205,244590,36.0,0.0,10.0,False
20990,52,TO DOWNTOWN,2021-08-23 21:33:59,35,NXIPBODY,0,5205,244590,136.0,1.0,21.0,False


In [118]:
sampled_distributions.loc[20786, 'disruptions'] = True
sampled_distributions[sampled_distributions["block_abbr"] == 5205]
fp = os.path.join('code/Scenarios/baseline/sampled_distributions_with_disruptions.pkl')
sampled_distributions.to_pickle(fp)

In [2]:
# All vehicles just read from this dataframe of pre-computed distributions
fp = os.path.join('code/Scenarios/baseline/sampled_distributions.pkl')
sampled_distributions = pd.read_pickle(fp)

In [4]:
sampled_distributions.query('zero_load_at_trip_end == 0')

,route_id,route_direction,scheduled_time,stop_sequence,stop_id_original,zero_load_at_trip_end,block_abbr,trip_id,travel_times,ons,loads,disruptions
0,3,FROM DOWNTOWN,2021-08-23 05:20:00,1,MCC5_5,0,300,245109,928.0,6.0,17.0,False
1,3,FROM DOWNTOWN,2021-08-23 05:22:36,2,6AVDEASN,0,300,245109,32.0,0.0,9.0,False
2,3,FROM DOWNTOWN,2021-08-23 05:23:17,3,6AVCHUSN,0,300,245109,92.0,0.0,10.0,False
3,3,FROM DOWNTOWN,2021-08-23 05:24:48,4,8ABROSN,0,300,245109,30.0,0.0,14.0,False
4,3,FROM DOWNTOWN,2021-08-23 05:25:15,5,BRO9AWF,0,300,245109,36.0,0.0,9.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
18,50,FROM DOWNTOWN,2021-08-23 23:39:18,20,ANNPREEF,0,9975,246345,28.0,0.0,4.0,False
19,50,FROM DOWNTOWN,2021-08-23 23:40:00,21,CXOANNEX,0,9975,246345,142.0,0.0,1.0,False
20,50,FROM DOWNTOWN,2021-08-23 23:41:45,22,CXONVLWS,0,9975,246345,78.0,0.0,4.0,False
21,50,FROM DOWNTOWN,2021-08-23 23:44:39,23,CXODVDSN,0,9975,246345,142.0,0.0,0.0,False


In [41]:
sampled_distributions.query("stop_sequence == 1 and travel_times <= 0")

,route_id,route_direction,scheduled_time,stop_sequence,stop_id_original,zero_load_at_trip_end,block_abbr,trip_id,travel_times,ons,loads,disruptions
0,4,TO DOWNTOWN,2021-08-23 11:31:00,1,19SSHESN,0,410,245759,0.0,0.0,8.0,False
0,5,FROM DOWNTOWN,2021-08-23 15:00:00,1,MCC5_5,0,502,245502,0.0,10.0,11.0,False
0,8,TO DOWNTOWN,2021-08-23 06:53:00,1,100OAKS,0,801,243603,0.0,1.0,1.0,False
0,29,FROM DOWNTOWN,2021-08-23 16:20:00,1,MCC5_8,0,1700,242163,0.0,5.0,17.0,False
0,23,TO DOWNTOWN,2021-08-23 22:42:00,1,DWMRT,0,2310,245846,0.0,2.0,13.0,False
0,25,SOUTHBOUND,2021-08-23 09:00:00,1,25ACLASN,0,2500,242092,0.0,0.0,0.0,False
0,25,SOUTHBOUND,2021-08-23 15:45:00,1,25ACLASN,0,2503,242101,0.0,0.0,2.0,False
0,28,FROM DOWNTOWN,2021-08-23 21:15:00,1,MCC4_21,0,2800,242140,0.0,2.0,7.0,False
0,56,FROM DOWNTOWN,2021-08-23 17:45:00,1,MCC4_16,0,5603,244997,0.0,13.0,17.0,False
0,93,LOOP,2021-08-23 15:47:00,1,21WE,0,9302,243423,0.0,1.0,1.0,False


In [43]:
a = np.array([0, 2, -1])
a[a>0]

array([2])

In [49]:
traveltime_dataset.query("route_id_direction == '93_LOOP' and block_abbr == 9302 and stop_sequence == 1")

time_to_next_stop
route_id_direction block_abbr stop_sequence stop_id_original time_window IsWeekend                                                   
93_LOOP            9302       1             21WE             61          0                                                   [1294.0]
                                                             62          0          [341.0, 338.0, 659.0, 906.0, 422.0, 415.0, 459...
                                                             63          0          [6.0, 0.0, 212.0, 8.0, 0.0, -1.0, -1.0, 143.0,...
                                                             64          0                             [989.0, 993.0, 1209.0, 1054.0]
                                                             65          0          [684.0, 690.0, 416.0, 694.0, 205.0, 675.0, 7.0...
                                                             66          0          [221.0, 0.0, 2.0, 0.0, 0.0, 0.0, 2.0, 3.0, 0.0...
                                                             68          0          [914.0, 652.0, 576.0, 944.0, 610.0, 907.0, 912...
                                                             69          0          [9.0, 0.0, 8.0, 167.0, 0.0, 1.0, 3.0, 3.0, 197...
                                                             71          0                                                    [114.0]